In [ ]:
!pip install timm
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
import mediapipe as mp
from PIL import Image, ImageOps
import numpy as np

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

def detect_and_crop_face(image):
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        image_np = np.array(image)
        results = face_detection.process(image_np)
        if results.detections:
            detection = results.detections[0]
            bbox = detection.location_data.relative_bounding_box
            ih, iw, _ = image_np.shape
            xmin = int(bbox.xmin * iw)
            ymin = int(bbox.ymin * ih)
            width = int(bbox.width * iw)
            height = int(bbox.height * ih)
            xmax = xmin + width
            ymax = ymin + height
            face = image.crop((xmin, ymin, xmax, ymax))
            return face
        else:
            return -1




In [ ]:
import os
path='/content/drive/MyDrive/TEST_DATA_SET'
anger_path=os.path.join(path,'anger')
happy_path=os.path.join(path,'happy')
panic_path=os.path.join(path,'panic')
sadness_path=os.path.join(path,'sadness')
label_path=os.path.join(path,'label (라벨링)')

In [ ]:
import json
test_df=pd.DataFrame(columns=['img_path','label','gender','age'])
i=0
for label_name in os.listdir(label_path):
    with open(os.path.join(label_path,label_name),'r',encoding='cp949') as f:
        file=json.load(f)
        for v in file:
            if v['filename'].split('.')[-1]=='jpeg':
                continue
            if v['gender']=='남':
                gender='남자'
            else:
                gender='여자'
            if v['faceExp_uploader']=='분노':
                label='anger'
            elif v['faceExp_uploader']=='기쁨':
                label='happy'
            elif v['faceExp_uploader']=='당황':
                label='panic'
            elif v['faceExp_uploader']=='슬픔':
                label='sadness'
            test_df.loc[i]=[os.path.join(path,label,v['filename']),label,gender,v['age']]
            i+=1
test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
expression_test_transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
age_test_transform=Compose([
    Resize((256,256)),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
gender_test_transform=Compose([
    Resize((256,256)),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
expression_model = torch.load("/content/drive/MyDrive/best_model.pt", map_location=device)
age_model = torch.load('/content/drive/MyDrive/swinv2_ages.pt', map_location=device)
gender_model = torch.load('/content/drive/MyDrive/swinv2_gender.pt', map_location=device)
expression_model.eval()
age_model.eval()
gender_model.eval()
expression_model.to(device)
age_model.to(device)
gender_model.to(device)

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error
label=['anger','happy','panic','sadness']
gender=['남자','여자']
expression_preds,expression_true_labels=[],[]
age_preds,age_true_labels=[],[]
gender_preds,gender_true_labels=[],[]
with torch.no_grad():
    for i in tqdm(range(len(test_df))):
        img_path=test_df.iloc[i]['img_path']
        img=Image.open(img_path).convert('RGB')
        img=ImageOps.exif_transpose(img)
        cropped_img=detect_and_crop_face(img)
        if cropped_img==-1:
            print(test_df.iloc[i]['img_path'])
            continue
        expression_img=expression_test_transform(cropped_img)
        age_img=age_test_transform(cropped_img)
        gender_img=gender_test_transform(cropped_img)
        expression_img=expression_img.unsqueeze(0).to(device)
        age_img=age_img.unsqueeze(0).to(device)
        gender_img=gender_img.unsqueeze(0).to(device)
        expression_pred=expression_model(expression_img)
        age_pred=age_model(age_img)
        gender_pred=gender_model(gender_img)
        expression_pred=label[expression_pred.argmax(1).detach().cpu().numpy().tolist()[0]]
        age_pred=age_pred.detach().cpu().item()
        gender_pred=gender[(torch.sigmoid(gender_pred) > 0.5).int().detach().cpu().numpy().tolist()[0][0]]
        expression_preds.append(expression_pred)
        expression_true_labels.append(test_df.iloc[i]['label'])
        age_preds.append(age_pred)
        age_true_labels.append(test_df.iloc[i]['age'])
        gender_preds.append(gender_pred)
        gender_true_labels.append(test_df.iloc[i]['gender'])


In [ ]:
expression_acc=accuracy_score(expression_true_labels,expression_preds)
age_mae = mean_absolute_error(age_true_labels, age_preds)
gender_acc=accuracy_score(gender_true_labels,gender_preds)

In [ ]:
print(f'expression_acc:{expression_acc}')
print(f'age_mae:{age_mae}')
print(f'gender_acc:{gender_acc}')